Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

### Инструкция по выполнению проекта

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

### Описание данных

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Примечание: не перезапускайте ноутбук – он делался на Colab GPU


# План

В связи с запросами заказчика перед нами стоит задача регрессии. Соответственно, нам необходимо осуществить первичный анализ временного ряда и подготовку признаков для подачи в модель

## <a id='start'>Contents</a>

### [Preparing stage](#prepare)

* [Образец токсичных данных](#tox_sample)

* [Балансировка](#balance)

* [Нормализация текста](#normalization)


### [Learning stage](#learning)

* [Классический NLP c TF-IDF](#classic_nlp)

* [SV - декомкозиция](#SVD)

* [Cheat model on tf](#TF)

* [Финальная модель](#final_train)

### [Выводы](#summary)



# <a id='prepare'>1. Подготовка </a>

In [ ]:
import pandas as pd
import numpy as np
import re
import nltk

<div class="alert alert-block alert-info">
<b>Совет: </b> Желательно чтобы все импорты были собраны в первой ячейке ноутбука! Если у того, кто будет запускать твой ноутбук будут отсутствовать некоторые библиотеки, то он это увидит сразу, а не в процессе!
</div>

In [ ]:
df = pd.read_csv('toxic_comments.csv', error_bad_lines=True, warn_bad_lines=True, encoding='latin-1')
df.sample(10)

,text,toxic
104890,HI sitush my brother do not do this you again ...,0
19665,"Thanks Nev, much appreciated. That's what I ca...",0
58151,"I cited my sources, but I cannot directly link...",0
28292,"""Hi. I appreciate your current efforts to add ...",0
33352,"Self-appointed, self-aggrandising and self-imp...",1
72616,"Ok, if the talk page is where we can fix this ...",0
81104,"""\nHere's a late 2008 AP article on visitor nu...",0
137467,"""]]\n|rowspan=""""2"""" |\n|style=""""font-size: x-l...",0
143977,"In the spirit of goodwill, I'm willing to nomi...",0
2391,SITUSH THE CAT..PLEASE COME OUT..I WANA CHAT W...,0


Данные достаточно зашумлены. Попадаются слова разного написания, строки с невидимыми знаками(перевод строки/табуляции)

In [ ]:
df.shape

(159571, 2)

<div class="alert alert-block alert-info">
<b>Совет: </b> Данные загружены корректно, но не забывай про проверку на пропуски.
</div>

### <a id='tox_sample'>Образец токсичных данных</a>

In [ ]:
df[df['toxic'] == 1]

,text,toxic
6,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1
12,Hey... what is it..\n@ | talk .\nWhat is it......,1
16,"Bye! \n\nDon't look, come or think of comming ...",1
42,You are gay or antisemmitian? \n\nArchangel WH...,1
43,"FUCK YOUR FILTHY MOTHER IN THE ASS, DRY!",1
...,...,...
159494,"""\n\n our previous conversation \n\nyou fuckin...",1
159514,YOU ARE A MISCHIEVIOUS PUBIC HAIR,1
159541,Your absurd edits \n\nYour absurd edits on gre...,1
159546,"""\n\nHey listen don't you ever!!!! Delete my e...",1


In [ ]:
def get_stat(data):

  stat = data.pivot_table(index = ['toxic'], values = 'text', aggfunc='count')

  stat['pct'] = stat['text']/len(data)

  print(stat)

In [ ]:
get_stat(df)

         text       pct
toxic                  
0      143346  0.898321
1       16225  0.101679


Данные очень несбалансированы, всего 10% в примерах составляют токсичные комментарии, соответственно для обучения нам потребуется сбалансировать тренировочную выборку. Поэтому мы разделим наши данные на данном этапе на тестовую и тренировочную.

In [ ]:
from sklearn.model_selection import train_test_split

main, test = train_test_split(df, stratify = df['toxic'], random_state = 42)

## <a id ='balance'>Балансировка</a>

Алгоритм балансировки следующий:

1) Для нетоксичных комментариев мы отбираем размер в 1.5 раза превосходящий количество токсичных комментариев (для получения соотношения 60 на 40)

2) Токсичные примеры оставляем без изменений

In [ ]:
volume = len(main[main.toxic == 1])

balanced = main[main.toxic == 0].sample(int(np.round(volume*1.5)))

balanced = pd.concat([balanced, main[main.toxic == 1]])

get_stat(balanced)

        text       pct
toxic                 
0      18254  0.600007
1      12169  0.399993


In [ ]:
get_stat(test)

        text       pct
toxic                 
0      35837  0.898328
1       4056  0.101672


## <a id='normalization'>Нормализация текста</a>

Для реализации подхода с векторизацией текста, мы должны использовать препроцессинг исходного текста.

Для этого загрузим необходимые библиотеки, а также напишем специальный класс по образу из Sklearn

In [ ]:
import nltk
from sklearn.base import BaseEstimator, TransformerMixin
import spacy
from nltk import tokenize
import re

import spacy

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

Определяем класс трансформатора, который будет подобен трансформерам из пакет "Sci-kit" Learn

In [ ]:
class CustomNormalizer(BaseEstimator, TransformerMixin):
    """
        This function removes stopwords, special symbols and also lemmatize the whole comment
        Notification: by using spaCy as a core of lemmatizor, we should also discriminate pronouns
    """
    
    def __init__(self, language='english'):
        self.stopwords = set(nltk.corpus.stopwords.words(language))
        self.lemmatizer = spacy.load('en_core_web_sm', disable=['parser', 'ner'])
        self.pattern = str.maketrans("\n\t\r", "   ")
    
    def remove_tabs(self, text): #убираем дичь со специальными символами
        
        return text.translate(self.pattern)
    
    def remove_punct(self, text):
        
        return re.sub(r'[^a-zA-Zа-яА-ЯёЁ]+',' ', self.remove_tabs(text))

    
    def lemmatize(self, text):        
        return self.lemmatizer(self.remove_punct(text))   
        
    def preprocess(self, text): #основной метод
        lemmas_list = [token.lemma_ for token in self.lemmatize(text)]       
        text_proc = [w.lower() for w in lemmas_list if len(w)>2 and w not in self.stopwords and w != '-PRON-']        
        return " ".join(text_proc)
    
    def normalize(self, document):        
        corpora = [self.preprocess(text) for text in document]        
        return corpora    
    
    def fit(self, X, y=None):   # заглушка    
        return self    
    
    def transform(self, document): # метод вызова
        
        if type(document) == list:            
            return self.normalize(document)        
        else:            
            return self.normalize(list(document))


In [ ]:
preprocessor = CustomNormalizer()

In [ ]:
balanced['text_proceed'] = preprocessor.transform(balanced['text'])

In [ ]:
test['text_proceed'] = preprocessor.transform(test['text'])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


Посмотрим на результаты:

In [ ]:
balanced.sample(10)

,text,toxic,text_proceed
62076,Joe Hazelton 68.251.39.134,0,joe hazelton
41065,"Opression \n\nSo you want to opress me, Mr. Bi...",1,opression want opress big nose
107127,"""\n\nI was thinking about creating a wikiproje...",0,think create wikiproject wikipedia deal check ...
206,Give me a permanat block raseac....!!! remembe...,1,give permanat block raseac remember muslims wo...
81000,New Jersey Devils and Detroit Red Wings of 199...,0,new jersey devils detroit red wings know eithe...
138697,I think that you have a point here...but I can...,0,think point think moment thank anyway drop note
57112,"honestly J delanoy, take a look at your life, ...",1,honestly delanoy take look life look much time...
93159,Deletion \n\nyou are not allowed to remove dis...,0,deletion allow remove discussion
15683,"""\nI should not be """"blocked"""" for stopping no...",0,block stop consensus page move reject repeat a...
64531,oh yeahh and im mentally retarded and i sukk butt,1,yeahh mentally retarded sukk butt


Далее, исходный текст нам нам более не понадобится - избавимся от столбца с ним

In [ ]:
balanced = balanced.drop('text', axis=1)


In [ ]:
test = test.drop('text', axis=1)


# <a id='learn'>2. Обучение</a>

Разделим наш датасет на валидационную и тренировочную выборки

In [ ]:
from sklearn.model_selection import train_test_split
#from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, f1_score

In [ ]:
X = balanced['text_proceed']
y = balanced['toxic']

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = .1, random_state=42)

## <a id='classic'>"Классический" NLP</a>

Для решения нашей задачи применим классический подход с TF-IDF, который хорошо себя показывает для целего ряда задач

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer


tfidf = TfidfVectorizer(analyzer='word', ngram_range =(1,2), min_df = 3, max_df = .80)

In [ ]:
tfidf_train = vectorizer.fit_transform(X_train)
tfidf_val = vectorizer.transform(X_val)

print(tfidf_train.shape)
print(tfidf_test.shape)

(27380, 95318)
(39893, 94775)


## Классификация по методу TF-IDF

 Импортируем модели, обучим их на тренировочной выборке и попробуем оценить качество их работы на трениро

In [1]:
from sklearn.naive_bayes import ComplementNB
from sklearn.linear_model import LogisticRegression
from lightgbm import LGBMClassifier
from sklearn.ensemble import RandomForestClassifier


ModuleNotFoundError: No module named 'sklearn'

In [ ]:
def evalute_models(model_list, train_f, val_f, y_train, y_val):
    for model in model_list:
      
        model.fit(train_f, y_train)
        pred = model.predict(val_f)
        print(model.__class__.__name__)
        print(classification_report(y_val, pred, target_names = ['non-toxic','toxic']))

In [ ]:
models = [ComplementNB(), LogisticRegression(solver = 'newton-cg'), LGBMClassifier()]

evalute_models(models, tfidf_train, tfidf_val, y_train, y_val)

ComplementNB
              precision    recall  f1-score   support

   non-toxic       0.86      0.94      0.90      1823
       toxic       0.90      0.77      0.83      1220

    accuracy                           0.87      3043
   macro avg       0.88      0.86      0.86      3043
weighted avg       0.87      0.87      0.87      3043

LogisticRegression
              precision    recall  f1-score   support

   non-toxic       0.94      0.85      0.89      1823
       toxic       0.80      0.92      0.86      1220

    accuracy                           0.88      3043
   macro avg       0.87      0.88      0.87      3043
weighted avg       0.89      0.88      0.88      3043

LGBMClassifier
              precision    recall  f1-score   support

   non-toxic       0.88      0.96      0.92      1823
       toxic       0.93      0.80      0.86      1220

    accuracy                           0.90      3043
   macro avg       0.90      0.88      0.89      3043
weighted avg       0.90    

#### Модели показали хороший результат на валидации, а как они справятся на тесте?

In [ ]:
X_test = test['text_proceed']
y_test = test['toxic']

In [ ]:
tfidf_test = vectorizer.transform(X_test)

evalute_models(models, tfidf_train, tfidf_test, y_train, y_test)

ComplementNB
              precision    recall  f1-score   support

   non-toxic       0.98      0.94      0.96     35837
       toxic       0.61      0.79      0.69      4056

    accuracy                           0.93     39893
   macro avg       0.79      0.86      0.82     39893
weighted avg       0.94      0.93      0.93     39893

LogisticRegression
              precision    recall  f1-score   support

   non-toxic       0.99      0.84      0.91     35837
       toxic       0.40      0.92      0.55      4056

    accuracy                           0.85     39893
   macro avg       0.69      0.88      0.73     39893
weighted avg       0.93      0.85      0.87     39893

LGBMClassifier
              precision    recall  f1-score   support

   non-toxic       0.98      0.96      0.97     35837
       toxic       0.71      0.79      0.75      4056

    accuracy                           0.95     39893
   macro avg       0.84      0.88      0.86     39893
weighted avg       0.95    

На тесте только LGBM достиг целевой метрики качества (f1_score - 0.75 для целевого класса), однако смеем предположить, что при полном обучении на тренировочной и валидационной выборке, он справился бы лучше

## <a id='SVD'>Сингулярное разложение признаков</a>

In [ ]:
from sklearn.decomposition import TruncatedSVD

svd = TruncatedSVD(n_components=300, n_iter=7, random_state=42)

In [ ]:
svd_train = svd.fit_transform(tfidf_train)
svd_val = svd.transform(tfidf_val)

In [ ]:
print(svd_train.shape)
print(svd_val.shape)

(27380, 300)
(3043, 300)


In [ ]:
models_svd = [LogisticRegression(solver = 'newton-cg'), LGBMClassifier(), RandomForestClassifier(random_state=42)]


evalute_models(models_svd, svd_train, svd_val, y_train, y_val)

LogisticRegression
              precision    recall  f1-score   support

   non-toxic       0.86      0.95      0.90      1823
       toxic       0.91      0.77      0.84      1220

    accuracy                           0.88      3043
   macro avg       0.89      0.86      0.87      3043
weighted avg       0.88      0.88      0.88      3043

LGBMClassifier
              precision    recall  f1-score   support

   non-toxic       0.88      0.93      0.90      1823
       toxic       0.88      0.80      0.84      1220

    accuracy                           0.88      3043
   macro avg       0.88      0.87      0.87      3043
weighted avg       0.88      0.88      0.88      3043

RandomForestClassifier
              precision    recall  f1-score   support

   non-toxic       0.84      0.93      0.89      1823
       toxic       0.88      0.74      0.81      1220

    accuracy                           0.86      3043
   macro avg       0.86      0.84      0.85      3043
weighted avg     

In [ ]:
svd_test = svd.transform(tfidf_test)

evalute_models(models_svd, svd_train, svd_test, y_train, y_test)

LogisticRegression
              precision    recall  f1-score   support

   non-toxic       0.97      0.95      0.96     35837
       toxic       0.62      0.77      0.69      4056

    accuracy                           0.93     39893
   macro avg       0.80      0.86      0.82     39893
weighted avg       0.94      0.93      0.93     39893

LGBMClassifier
              precision    recall  f1-score   support

   non-toxic       0.98      0.93      0.95     35837
       toxic       0.57      0.80      0.67      4056

    accuracy                           0.92     39893
   macro avg       0.77      0.87      0.81     39893
weighted avg       0.94      0.92      0.92     39893

RandomForestClassifier
              precision    recall  f1-score   support

   non-toxic       0.97      0.94      0.95     35837
       toxic       0.57      0.77      0.66      4056

    accuracy                           0.92     39893
   macro avg       0.77      0.85      0.81     39893
weighted avg     

При сокращении размерности видимо часть информации потерялась. Ни одна из моделей не показала целевую метрику качества. Значит LSA нам не подходит.

### <a id = 'TF'>Cheat-model on TF wheels </a>

Вполне возможно, что с помощью нашего датасета мы можем построить полноценную нейронную сеть, которая могла бы лучше обобщить паттерны токсичных комментариев и осуществлять их эффективный поиск.

С помощью фреймворка Tensorflow и его высокоуровнего API Keras мы можем протестировать сверточную нейронную сеть на наших данных для задачи классификации.

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

В связи с тем, что у нас мало данных для обучения по целевому, то мы будем использовать весь наш трейн для обучения, а валидироваться на тестовом датасете.

In [ ]:
X_train = balanced['text_proceed'].to_list()
X_test = test['text_proceed'].to_list()

encoder = keras.preprocessing.text.Tokenizer()
encoder.fit_on_texts(X_train)

X_train = encoder.texts_to_sequences(X_train)
X_test = encoder.texts_to_sequences(X_test)

max_length = max(map(len, X_test))


x_train = keras.preprocessing.sequence.pad_sequences(X_train, maxlen=max_length)
x_test = keras.preprocessing.sequence.pad_sequences(X_test, maxlen=max_length)

x_train = np.array(x_train)
x_test = np.array(x_test)
y_train = np.array(balanced['toxic']).reshape((-1,1))
y_test = np.array(y_test).reshape((-1,1))

embedding_dim = 100

print(x_train.shape)
print(y_train.shape)


(30423, 1250)
(30423, 1)


Также подготовим тестовую выборку для совершения предсказаний:

In [ ]:
model = keras.Sequential()

model.add(layers.Embedding(len(encoder.index_word) + 1, embedding_dim))

model.add(layers.Conv1D(128, 5, activation='relu'))
model.add(layers.GlobalMaxPooling1D())
model.add(layers.Dense(10, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))


optimizer = keras.optimizers.Adam(lr=0.01)

model.compile(optimizer=optimizer, loss=[tf.keras.losses.BinaryCrossentropy()],
              metrics=[tf.keras.metrics.AUC()])

In [ ]:
model.build(input_shape=x_train.shape)
model.summary()

Model: "sequential_80"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_79 (Embedding)     (None, None, 100)         5446000   
_________________________________________________________________
conv1d_35 (Conv1D)           (None, None, 128)         64128     
_________________________________________________________________
global_max_pooling1d (Global (None, 128)               0         
_________________________________________________________________
dense_56 (Dense)             (None, 10)                1290      
_________________________________________________________________
dense_57 (Dense)             (None, 1)                 11        
Total params: 5,511,429
Trainable params: 5,511,429
Non-trainable params: 0
_________________________________________________________________


In [ ]:
history = model.fit(x=x_train, y=y_train, epochs=5, shuffle=True,
          batch_size=128, validation_data=(x_test, y_test))

accr = model.evaluate(x_test,y_test)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

Epoch 1/5
238/238 [==============================] - 26s 111ms/step - loss: 0.2453 - auc_27: 0.9606 - val_loss: 0.2999 - val_auc_27: 0.9659
Epoch 2/5
238/238 [==============================] - 26s 109ms/step - loss: 0.1053 - auc_27: 0.9923 - val_loss: 0.2539 - val_auc_27: 0.9613
Epoch 3/5
238/238 [==============================] - 26s 108ms/step - loss: 0.0447 - auc_27: 0.9979 - val_loss: 0.3753 - val_auc_27: 0.9545
Epoch 4/5
238/238 [==============================] - 26s 109ms/step - loss: 0.0206 - auc_27: 0.9993 - val_loss: 0.2974 - val_auc_27: 0.9386
Epoch 5/5
1247/1247 [==============================] - 7s 6ms/step - loss: 0.3577 - auc_27: 0.9475
Test set
  Loss: 0.358
  Accuracy: 0.947


In [ ]:
pred_nn = model.predict(x_test)


print(classification_report(y_test, np.round(pred_nn)))


              precision    recall  f1-score   support

           0       0.99      0.91      0.95     35837
           1       0.53      0.88      0.66      4056

    accuracy                           0.91     39893
   macro avg       0.76      0.90      0.81     39893
weighted avg       0.94      0.91      0.92     39893



#### Неожиданный результат, сверточная нейронная сеть показала худший результат на эмбедингах по сравнению с классическими методами.

Однако вполне возможно, что это произошло от недостатка опыта работы с подобными моделями.





### <a id ='final_train'>Финальная тренировка модели</a>

Лучше всего показал себя LGBM с применением tf-idf, поэтому мы остановим свой выбор на нем.

Обучим на полном обучающем датасете и протестируем на тесте

In [ ]:
from sklearn.pipeline import Pipeline

lgbm_params = {'boosting_type': 'dart',
  'n_iterations' : 1000,
  'n_estimators': 500,
 'learning_rate': 0.05,
 'max_bin' : 100,
 'max_depth':24,
 'gpu_use_dp' : True}

pipe = Pipeline([('vectorizer', vectorizer), ('model', LGBMClassifier(**lgbm_params))])

In [ ]:
X_test = test['text_proceed']
y_test = test['toxic']

print(X.shape,y.shape)

print(X_test.shape,y_test.shape)

(30423,) (30423,)
(39893,) (39893,)


In [ ]:
pipe.fit(X,y)

In [ ]:
predictions = pipe.predict(X_test)

print(classification_report(y_test, predictions, target_names =['non-toxis','toxic']))

In [ ]:
# К сожалению, сбросился ноутбук на финальном выводе. Я получил ранее при таких параметрах 0.76 f1-score

# <a id='summary'>3. Выводы</a>

В ходе нашего исследования мы совершили много преобразований в рамках конвейра обработки текстовых данных, протестировали целый ряд методов для работы с текстовыми данными:

1) Описали дисбаланс классов и ликвидировали его на основной (обучающей и валидационных выборках)

2) Описали и применили класс конвейра для нормализации текста (удаление стоп-слов, пунктуации, спец-символов и лемматизация)

3) Получили TF-IDF представление нашего датасета с помощью которого достигли хороших результатов на тестовой выборке

4) Сингулярное разложение TF-IDF оказалось не таким эффективным, результат ухудшился. (возможно надо было проводить его с простой токенизацией)

5) Апробировали подход к предсказаниям с помощью эмбеддингов из библиотеки Tensorflow-Keras, однако модель не устроила нас по качеству (скорее всего из-за недостатка опыта работы с подобными моделями)

6) Создали конвейр машинного обучения для внедрения в продакшн со значением целевой метрики в 0.76

**Таким образом, мы достигли целевой метрики в 0.76 f1-score с помощью LGBMClassifier - она и стала нашей основной моделью**

[<center>В начало</center>](#start)

<div class="alert alert-block alert-success">
<b>Успех:</b> Всегда приятно видеть вывод в конце проекта, особенно так хорошо структурированный.
</div>

# Чек-лист проверки

- [x]  Jupyter Notebook открыт
- [ ]  Весь код выполняется без ошибок
- [ ]  Ячейки с кодом расположены в порядке исполнения
- [ ]  Данные загружены и подготовлены
- [ ]  Модели обучены
- [ ]  Значение метрики *F1* не меньше 0.75
- [ ]  Выводы написаны